In [2]:
from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import Dataset
import os
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.transform = transform
        self.images = []
        self.labels = []
        for filename in os.listdir(directory):
            if filename.endswith('.jpeg'):
                self.images.append(os.path.join(directory, filename))


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        if self.transform:
            image = self.transform(image)
        return image

# Use the transform while creating the dataset
transform = Compose([
    Resize((224, 224)),  # Resize the image to 224x224 pixels
    ToTensor(),  # Convert the image to PyTorch Tensor data type
])

dataset = CustomDataset('data/gtsrb/test_224', transform=transform)

In [6]:
dataset[2].shape

torch.Size([3, 224, 224])

In [1]:
import numpy as np
import os
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
def compute_mean_std_per_channel(directories):
    num_images = 0
    sum_image = np.zeros(3)
    sum_squared_image = np.zeros(3)

    for directory in directories:
        for filename in os.listdir(directory):
            if filename.endswith('.jpeg') or filename.endswith('.JPEG'):
                image = Image.open(os.path.join(directory, filename))
                image = ToTensor()(image)
                for i in range(3):  # Loop over the 3 color channels
                    sum_image[i] += image[i, :, :].mean()
                    sum_squared_image[i] += image[i, :, :].std()
                num_images += 1

    mean = sum_image / num_images
    std = np.sqrt(sum_squared_image / num_images - mean ** 2)

    return mean, std

# Compute mean and std for the combined datasets
mean, std = compute_mean_std_per_channel(['data/imagenet/train'])

print(f'Combined dataset: mean = {mean}, std = {std}')

Combined dataset: mean = [nan nan nan], std = [nan nan nan]


/tmp/ipykernel_1443649/2205030506.py:20: RuntimeWarning: invalid value encountered in divide
  mean = sum_image / num_images
/tmp/ipykernel_1443649/2205030506.py:21: RuntimeWarning: invalid value encountered in divide
  std = np.sqrt(sum_squared_image / num_images - mean ** 2)


In [1]:
import os
import glob
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import torch
# 创建一个将图像转换为张量的转换
transform = transforms.ToTensor()

# 获取所有JPEG图像的路径
image_files = glob.glob(os.path.join('data/gtsrb/train', '*.[jJ][pP][eE][gG]'))

# 初始化列表以保存所有图像的张量
images = []

for image_file in image_files:
    # 打开图像并转换为张量
    image = Image.open(image_file)
    image_tensor = transform(image)
    images.append(image_tensor)

# 将所有图像张量堆叠到一个张量中
images = torch.stack(images)

# 计算mean和std
mean = images.mean([0, 2, 3])
std = images.std([0, 2, 3])

print('mean:', mean)
print('std:', std)

mean: tensor([0.3389, 0.3117, 0.3204])
std: tensor([0.2708, 0.2588, 0.2618])


In [2]:
import os
import random
from shutil import copyfile, rmtree

def select_quarter_images(directory, output_directory):
    # Create a dictionary to hold the images for each class
    class_images = {}

    # Loop over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.jpeg'):
            # Extract class from filename
            class_label = int(filename.split('_')[3].replace('.jpeg', '')[1:-1])
            # Add image to the list for this class
            class_images.setdefault(class_label, []).append(filename)

    # Select a quarter of the images for each class
    for class_label, images in class_images.items():
        selected_images = random.sample(images, len(images) // 5)
        # Copy selected images to output directory
        for image in selected_images:
            copyfile(os.path.join(directory, image), os.path.join(output_directory, image))

rmtree('data/gtsrb/selected_train_224', ignore_errors=True)
rmtree('data/stl10/selected_train_224', ignore_errors=True)
rmtree('data/svhn/selected_train_224', ignore_errors=True)

os.makedirs('data/gtsrb/selected_train_224', exist_ok=True)
os.makedirs('data/stl10/selected_train_224', exist_ok=True)
os.makedirs('data/svhn/selected_train_224', exist_ok=True)
# Select a quarter of the images from each class

select_quarter_images('data/gtsrb/train_224', 'data/gtsrb/selected_train_224')
select_quarter_images('data/stl10/train_224', 'data/stl10/selected_train_224')
select_quarter_images('data/svhn/train_224', 'data/svhn/selected_train_224')

In [6]:
import numpy as np
input=np.load('data/stl10/train_unlabeled.npz')
input['x'].shape

(105000, 32, 32, 3)

In [10]:
input['y'].shape

(5000, 1)

In [8]:
input=np.load('data/stl10/train.npz')
input['x'].shape

(5000, 32, 32, 3)